In [ ]:
from keras.preprocessing.image import ImageDataGenerator


from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet50

import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

In [ ]:
trainImagePath = './dataSet/splitImages/train'
validationImagePath = './dataSet/splitImages/val'
testImagePath = './dataSet/splitImages/test'

batchSize = 32
imageWidth = 224
imageHeight = 224
imageChannel = 3


### 이미지 데이터 생성

ImageDataGenerator (train, validation)

In [ ]:
trainDataGen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 30,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=20,
                                  zoom_range=0.3,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  brightness_range=[0.7, 1.3],
                                  fill_mode='nearest',
                                  )

train set

In [ ]:
trainGenSet = trainDataGen.flow_from_directory(
    trainImagePath,
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical',
    shuffle=True
)

validation set

In [ ]:
validationGenSet = trainDataGen.flow_from_directory(
    validationImagePath,
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical',
)

ImageDataGenerator (train, validation)

In [ ]:
testDataGen = ImageDataGenerator(rescale=1./255)

test set

In [ ]:
testGenSet = testDataGen.flow_from_directory(
    testImagePath,
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical'
)

In [ ]:
model = tf.keras.Sequential([
  ResNet50(include_top=False, weights='imagenet', input_shape=(imageWidth, imageHeight, imageChannel)),
  GlobalAveragePooling2D(),
  Dropout(rate=0.0),
  Dense(128, activation='relu' ),
  Dense(44, activation='softmax'),
])

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
checkpointPath = './checkpoints/epoch_{epoch:03d}.ckpt'

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                        checkpointPath, monitor='val_loss', verbose=0, save_best_only=True,
                        save_weights_only=True, mode='auto', save_freq='epoch'
                    )


In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=5, 
                             )

In [ ]:
epochs = 32
history = model.fit(
    trainGenSet,
    epochs=epochs,
    steps_per_epoch=trainGenSet.samples / epochs,
    validation_data=validationGenSet,
    validation_steps=validationGenSet.samples / epochs,
    callbacks=[checkpoint,earlyStopping],
    verbose=2
)

In [ ]:
scores = model.evaluate(testGenSet)
print(scores)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save_weights('./models/resnetModel')